Source: https://github.com/philschmid/huggingface-sagemaker-workshop-series/blob/main/workshop_1_getting_started_with_amazon_sagemaker/lab_1_default_training.ipynb

In [1]:
%%capture
!pip install "sagemaker>=2.250.0" "transformers>=4.55.2" "datasets==3.6.0"
# "datasets>=4.0.0" removed the dataset scripts functionality, which made a lot of datasets incompatible

In [2]:
import sagemaker
import sagemaker.huggingface

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [3]:
import sagemaker

sess = sagemaker.Session()
# sagemaker session bucket -> used for uploading data, models and logs
# sagemaker will automatically create this bucket if it not exists
sagemaker_session_bucket=None
if sagemaker_session_bucket is None and sess is not None:
    # set to default bucket if a bucket name is not given
    sagemaker_session_bucket = sess.default_bucket()

role = sagemaker.get_execution_role()
sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)

print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {sess.default_bucket()}")
print(f"sagemaker session region: {sess.boto_region_name}")

sagemaker role arn: arn:aws:iam::762595428873:role/SageMakerExecutionRole
sagemaker bucket: sagemaker-eu-west-2-762595428873
sagemaker session region: eu-west-2


In [4]:
from datasets import load_dataset
from transformers import AutoTokenizer

# tokenizer used in preprocessing
tokenizer_name = 'distilbert-base-uncased'

# dataset used
dataset_name = 'philschmid/emotion'

# s3 key prefix for the data
s3_prefix = 'samples/datasets/emotion'

In [5]:
# download tokenizer
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)

# tokenizer helper function
def tokenize(batch):
    return tokenizer(batch['text'], padding='max_length', truncation=True)

# load dataset
train_dataset, test_dataset = load_dataset(dataset_name, split=['train', 'test'], trust_remote_code=True)

# tokenize dataset
train_dataset = train_dataset.map(tokenize, batched=True)
test_dataset = test_dataset.map(tokenize, batched=True)

# set format for pytorch
train_dataset =  train_dataset.rename_column("label", "labels")
train_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])
test_dataset = test_dataset.rename_column("label", "labels")
test_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])

dataset_infos.json: 0.00B [00:00, ?B/s]

The repository for philschmid/emotion contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/philschmid/emotion.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N]  y


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:9                                                                                    │
│                                                                                                  │
│    6 │   return tokenizer(batch['text'], padding='max_length', truncation=True)                  │
│    7                                                                                             │
│    8 # load dataset                                                                              │
│ ❱  9 train_dataset, test_dataset = load_dataset(dataset_name, split=['train', 'test'])           │
│   10                                                                                             │
│   11 # tokenize dataset                                                                          │
│   12 train_dataset = train_dataset.map(tokenize, batched=True)                                   │
│                                                                                                  │
│ /home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/datasets/load.py:2062 in │
│ load_dataset                                                                                     │
│                                                                                                  │
│   2059 │   )                                                                                     │
│   2060 │                                                                                         │
│   2061 │   # Create a dataset builder                                                            │
│ ❱ 2062 │   builder_instance = load_dataset_builder(                                              │
│   2063 │   │   path=path,                                                                        │
│   2064 │   │   name=name,                                                                        │
│   2065 │   │   data_dir=data_dir,                                                                │
│                                                                                                  │
│ /home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/datasets/load.py:1817 in │
│ load_dataset_builder                                                                             │
│                                                                                                  │
│   1814 │   │   │   error_msg += f'\nFor example `data_files={{"train": "path/to/data/train/*.{e  │
│   1815 │   │   raise ValueError(error_msg)                                                       │
│   1816 │                                                                                         │
│ ❱ 1817 │   builder_cls = get_dataset_builder_class(dataset_module, dataset_name=dataset_name)    │
│   1818 │   # Instantiate the dataset builder                                                     │
│   1819 │   builder_instance: DatasetBuilder = builder_cls(                                       │
│   1820 │   │   cache_dir=cache_dir,                                                              │
│                                                                                                  │
│ /home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/datasets/load.py:249 in  │
│ get_dataset_builder_class                                                                        │
│                                                                                                  │
│    246 │   │   if dataset_module.importable_file_path                                            │
│    247 │   │   else nullcontext()                                                                │
│    248 │   ):                                                                                    │
│ ❱  249 │   │   builder_cls = import_main_class(dataset_modu